In [ ]:
# First, install TensorFlow
!pip install tensorflow==2.9.1

# Then import the required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
   conda install tensorflow
   

In [ ]:
# Load data prepared in Module 3
df = pd.read_csv(
    "final_feature_engineered_data.csv",
    index_col=0,
    parse_dates=True
)

df.head()


In [ ]:
# For LSTM, we will predict Energy_Scaled using past Energy_Scaled only
energy_values = df['Energy_Scaled'].values.reshape(-1, 1)

energy_values[:5]


In [ ]:
# Function to create sequences for LSTM
def create_sequences(data, time_steps=24):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:i + time_steps])
        y.append(data[i + time_steps])
    return np.array(X), np.array(y)

# Define number of past hours to look at
TIME_STEPS = 24   # 24 hours

# Create sequences
X, y = create_sequences(energy_values, TIME_STEPS)

X.shape, y.shape


In [ ]:
# Train-Test split (time-series safe)
split_ratio = 0.8
split_index = int(len(X) * split_ratio)

X_train = X[:split_index]
X_test  = X[split_index:]

y_train = y[:split_index]
y_test  = y[split_index:]

X_train.shape, X_test.shape


In [ ]:
# Build LSTM model
model = Sequential()

model.add(LSTM(50, activation='tanh', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1))

# Compile model
model.compile(optimizer='adam', loss='mse')

model.summary()


In [ ]:
# Early stopping to avoid overfitting
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=30,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1
)


In [ ]:
# Predict on test data
y_pred = model.predict(X_test)

y_pred[:5]


In [ ]:
# Calculate evaluation metrics for LSTM
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("LSTM Mean Absolute Error (MAE):", mae)
print("LSTM Root Mean Squared Error (RMSE):", rmse)


In [ ]:
plt.figure(figsize=(10, 4))

plt.plot(y_test[:100], label='Actual Energy', linewidth=2)
plt.plot(y_pred[:100], label='Predicted Energy', linestyle='--')

plt.title("Actual vs Predicted Energy Consumption (LSTM)")
plt.xlabel("Time Steps")
plt.ylabel("Energy (Scaled)")
plt.legend()
plt.show()
